This notebook explains how to use the diffthermo package to fit OCV models.

First, install the `diffthermo` package according to the README file. 

In [ ]:
conda create -n diffthermo python=3.11  torch=2.0.0 numpy=1.24 matplotlib pandas
conda activate diffthermo 
git clone https://github.com/BattModels/Diffthermo_OCV_paper.git
cd Diffthermo_OCV_paper
pip install .

To fit the OCV model, you need a csv file which contains the OCV VS SoC data. The first column of the file is SoC data, the second column is the OCV value. The OCV VS SoC data may come from experimental results or digitizing the results from figures of existing literature that contains OCV VS SoC information. 

In [1]:
import pandas as pd 
datafile_name = 'graphite.csv' # this is the csv file that you prepard, it should contain OCV VS SoC data
df = pd.read_csv(datafile_name,header=None)
print(df)

           0         1
0   0.020261  0.493823
1   0.021986  0.485258
2   0.022533  0.477210
3   0.023329  0.466564
4   0.024362  0.456673
..       ...       ...
94  0.918431  0.059359
95  0.930202  0.052110
96  0.939606  0.044030
97  0.947238  0.035824
98  0.954031  0.028318

[99 rows x 2 columns]


Now, you simply need to feed the prepared csv file that contains OCV VS SoC data into the diffthermo package. 

Put the name of your csv file as `datafile_name`. `number_of_Omegas` is a hyperparameter that you should choose. I suggest to start with small numbers (3 or 4 as examples), and increase the number if the fitted model does not look good. The `total_training_epochs` specifies how many round of updates the parameters (U0 and Omegas) will have. You should set the number accordingly, so that the loss value is minimized. I recommend the learning_rate to be 1000.0 (since the value of Omegas are usually large), you can leave this value unchanged. `loss_threshold` controls the minimum value of loss, below which the fitting process will automatically stop. 

A quick note: if your fitted results does not look good, TRY adjusting `G0_rand_range` and `Omegas_rand_range` in `train`. These two parameters control the initial guess of G0 and Omegas. Usually for an anode material, G0_rand_range=[-10*5000,-5*5000], 
 Omegas_rand_range=[-10*100,10*100] work well, and for cathode material, G0_rand_range=[-100*5000,-50*5000], Omegas_rand_range=[-100*100,100*100] work. 

Now let's just run the model and see what's going to happen:

In [2]:
from diffthermo.utils import train
params_list = train(datafile_name='graphite.csv', 
                        number_of_Omegas=8, 
                        learning_rate = 1000.0, 
                        total_training_epochs = 8000,
                        loss_threshold = 0.01,
                        G0_rand_range=[-10*5000,-5*5000], 
                        Omegas_rand_range=[-10*100,10*100])

Epoch   0  Loss 41.9855     Omega0 213.6310 Omega1 -1206.9720 Omega2 -91.9052 Omega3 -728.9696 Omega4 -217.9158 Omega5 -274.9674 Omega6 -563.9100 Omega7 -183.9645 G0 -27780.0020       
Epoch   1  Loss 36.3292     Omega0 1191.6555 Omega1 -2161.7571 Omega2 -822.0188 Omega3 -1702.0520 Omega4 -1092.1130 Omega5 -1258.5898 Omega6 -1488.5581 Omega7 -1173.5958 G0 -26781.3203       
Epoch   2  Loss 31.4385     Omega0 2181.1958 Omega1 -3011.0259 Omega2 -955.6142 Omega3 -2619.2085 Omega4 -1634.7057 Omega5 -2211.2515 Omega6 -2220.1101 Omega7 -2144.8508 G0 -25785.1816       
Epoch   3  Loss 27.1423     Omega0 2934.0532 Omega1 -3715.6543 Omega2 -697.5271 Omega3 -3465.0835 Omega4 -1811.4561 Omega5 -3126.9177 Omega6 -2691.4194 Omega7 -3094.7507 G0 -24793.6426       
Epoch   4  Loss 23.3154     Omega0 2955.0129 Omega1 -4248.0239 Omega2 -286.0047 Omega3 -4195.3350 Omega4 -1884.1201 Omega5 -3960.6333 Omega6 -3155.4475 Omega7 -3982.9243 G0 -23809.1484       
Epoch   5  Loss 19.8336     Omega0 2691.2368 Om

The loss value is decreasing in each epoch. Since we set the `total_training_epochs` to be 200, the fitting process stops at 200 epochs. You can adjust this number if you want to further minimize the loss value in order to get a more accurate OCV model.

Now, we have succesfully fitted an OCV model for graphite. To access your fitted model, call `write_ocv_functions` function:

In [3]:
from diffthermo.utils import write_ocv_functions
write_ocv_functions(params_list)

Found 3 phase coexistence region(s):
From x=0.5454377532005310 to x=0.9029746651649475
From x=0.9098722934722900 to x=0.8019378781318665
From x=0.2783287167549133 to x=0.4735463261604309





 Fitting Complete.

Fitted OCV function written in PyBaMM function (copy and paste readay!):

###################################

import numpy as np
import pybamm
from pybamm import exp, log, tanh, constants, Parameter, ParameterValues

def fitted_OCP(sto):
    _eps = 1e-7
    # rk params
    G0 = -12250.294922 # G0 is the pure substance gibbs free energy 
    Omega0 = -2376.557373 
    Omega1 = -4394.042969 
    Omega2 = 13664.444336 
    Omega3 = -52024.406250 
    Omega4 = -42901.992188 
    Omega5 = 138761.125000 
    Omega6 = 18488.441406 
    Omega7 = -117369.820312 
    Omegas =[Omega0, Omega1, Omega2, Omega3, Omega4, Omega5, Omega6, Omega7]
    # phase boundary 0
    x_alpha_0 = 0.5454377532005310
    x_beta_0 = 0.9029746651649475
    mu_coex_0 = -7666.7456054687500000
    is_outside_misc

You can directly copy & paste the fitted pybamm OCV function as printed in the terminal, or you can find the function in `fitted_ocv_functions.py`  file under the same directory where you put your csv file in. 